In [1]:
import pandas as pd
import json
import numpy as np
from elasticsearch import Elasticsearch

es = Elasticsearch(['localhost'])

## Dataset without Threat

In [2]:
#res = es.get(index="gebd-tracks-2018.12.07", doc_type='doc')
#res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={'query':{'match_all':{}}, "size": 10000})
res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={ \
            "query": { \
                "bool": { \
                    "must": [ \
                      { \
                        "match_phrase": { \
                          "log_name": { \
                            "query": "Microsoft-Windows-Sysmon/Operational" \
                          } \
                        } \
                      }, \
                      { \
                        "match_phrase": { \
                          "task": { \
                            "query": "Process Create (rule: ProcessCreate)" \
                          } \
                        } \
                      } \
                    ], \
                    "filter": [ \
                      { \
                        "bool": { \
                          "must_not": { \
                            "multi_match": { \
                              "type": "best_fields", \
                              "query": "rinetd_exploit", \
                              "lenient": True \
                            } \
                          } \
                        } \
                      }, \
                      { \
                        "match_all": {} \
                      } \
                    ] \
                } \
            }, "size": 10000})

#print(res['hits'])

#df = pd.DataFrame.from_dict(res['hits']['hits'])
from pandas.io.json import json_normalize
df_logs_without_threat = json_normalize(res['hits']['hits'])
pd.set_option('display.max_columns', None)

print("--- Column names and shape LOGS", df_logs_without_threat.columns, df_logs_without_threat.shape)
print(df_logs_without_threat.head())
print("SHAPE:", df_logs_without_threat.shape)

--- Column names and shape LOGS Index(['_id', '_index', '_score', '_source.@timestamp',
       '_source.beat.hostname', '_source.beat.name', '_source.beat.version',
       '_source.computer_name', '_source.event_data.CommandLine',
       '_source.event_data.Company', '_source.event_data.CurrentDirectory',
       '_source.event_data.Description', '_source.event_data.FileVersion',
       '_source.event_data.Hashes', '_source.event_data.Image',
       '_source.event_data.IntegrityLevel', '_source.event_data.LogonGuid',
       '_source.event_data.LogonId', '_source.event_data.ParentCommandLine',
       '_source.event_data.ParentImage',
       '_source.event_data.ParentProcessGuid',
       '_source.event_data.ParentProcessId', '_source.event_data.ProcessGuid',
       '_source.event_data.ProcessId', '_source.event_data.Product',
       '_source.event_data.TerminalSessionId', '_source.event_data.User',
       '_source.event_data.UtcTime', '_source.event_id', '_source.host.name',
       '_sour

In [3]:
df_logs_without_threat["_source.event_data.CommandLine"] = df_logs_without_threat["_source.event_data.CommandLine"].str.lower()
df_logs_without_threat["_source.event_data.CommandLine"] = df_logs_without_threat["_source.event_data.CommandLine"].str.replace("\"","")
df_logs_without_threat["_source.event_data.CurrentDirectory"] = df_logs_without_threat["_source.event_data.CurrentDirectory"].str.lower()


In [4]:
df_logs = df_logs_without_threat

### Number of unique command lines

In [5]:
len(np.unique(df_logs["_source.event_data.ParentCommandLine"]))

32

## Dataset with Threat

In [6]:
#res = es.get(index="gebd-tracks-2018.12.07", doc_type='doc')
#res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={'query':{'match_all':{}}, "size": 10000})
res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={ \
            "query": {
                "bool": {
                    "must": [
                      {
                        "match_phrase": {
                          "log_name": {
                            "query": "Microsoft-Windows-Sysmon/Operational"
                          }
                        }
                      },
                      {
                        "match_phrase": {
                          "task": {
                            "query": "Process Create (rule: ProcessCreate)"
                          }
                        }
                      }
                    ]
                }
            }, "size": 10000})

#print(res['hits'])

#df = pd.DataFrame.from_dict(res['hits']['hits'])
from pandas.io.json import json_normalize
df_logs_with_threat = json_normalize(res['hits']['hits'])
pd.set_option('display.max_columns', None)

print("--- Column names and shape LOGS", df_logs_with_threat.columns, df_logs_with_threat.shape)
print(df_logs_with_threat.head())
print("SHAPE:", df_logs_with_threat.shape)



--- Column names and shape LOGS Index(['_id', '_index', '_score', '_source.@timestamp',
       '_source.beat.hostname', '_source.beat.name', '_source.beat.version',
       '_source.computer_name', '_source.event_data.CommandLine',
       '_source.event_data.Company', '_source.event_data.CurrentDirectory',
       '_source.event_data.Description', '_source.event_data.FileVersion',
       '_source.event_data.Hashes', '_source.event_data.Image',
       '_source.event_data.IntegrityLevel', '_source.event_data.LogonGuid',
       '_source.event_data.LogonId', '_source.event_data.ParentCommandLine',
       '_source.event_data.ParentImage',
       '_source.event_data.ParentProcessGuid',
       '_source.event_data.ParentProcessId', '_source.event_data.ProcessGuid',
       '_source.event_data.ProcessId', '_source.event_data.Product',
       '_source.event_data.TerminalSessionId', '_source.event_data.User',
       '_source.event_data.UtcTime', '_source.event_id', '_source.host.name',
       '_sour

In [7]:
df_logs_with_threat["_source.event_data.CommandLine"] = df_logs_with_threat["_source.event_data.CommandLine"].str.lower()
df_logs_with_threat["_source.event_data.CommandLine"] = df_logs_with_threat["_source.event_data.CommandLine"].str.replace("\"","")
df_logs_with_threat["_source.event_data.CurrentDirectory"] = df_logs_with_threat["_source.event_data.CurrentDirectory"].str.lower()

In [8]:
df_logs = df_logs_with_threat

In [9]:
df_logs.describe()

,_score,_source.event_id,_source.process_id,_source.thread_id,_source.version
count,266.000000,266.0,266.000000,266.000000,266.0
mean,2.385393,1.0,2896.706767,4690.150376,5.0
std,0.082836,0.0,164.863936,30.136418,0.0
min,2.284866,1.0,2624.000000,4672.000000,5.0
25%,2.284866,1.0,2624.000000,4672.000000,5.0
50%,2.399398,1.0,2996.000000,4672.000000,5.0
75%,2.486658,1.0,2996.000000,4740.000000,5.0
max,2.486658,1.0,2996.000000,4740.000000,5.0


## First try with User and Company

In [10]:
#df_logs = df_logs[["_source.event_data.User", "_source.event_data.CurrentDirectory", "_source.event_data.CommandLine"]]
df_logs = df_logs[["_source.event_data.User", "_source.event_data.Company"]]

In [11]:
df_logs_with_threat[df_logs_with_threat["_source.user.type"] != "User"]

,_id,_index,_score,_source.@timestamp,_source.beat.hostname,_source.beat.name,_source.beat.version,_source.computer_name,_source.event_data.CommandLine,_source.event_data.Company,_source.event_data.CurrentDirectory,_source.event_data.Description,_source.event_data.FileVersion,_source.event_data.Hashes,_source.event_data.Image,_source.event_data.IntegrityLevel,_source.event_data.LogonGuid,_source.event_data.LogonId,_source.event_data.ParentCommandLine,_source.event_data.ParentImage,_source.event_data.ParentProcessGuid,_source.event_data.ParentProcessId,_source.event_data.ProcessGuid,_source.event_data.ProcessId,_source.event_data.Product,_source.event_data.TerminalSessionId,_source.event_data.User,_source.event_data.UtcTime,_source.event_id,_source.host.name,_source.level,_source.log_name,_source.message,_source.opcode,_source.process_id,_source.provider_guid,_source.record_number,_source.source_name,_source.task,_source.thread_id,_source.type,_source.user.domain,_source.user.identifier,_source.user.name,_source.user.type,_source.version,_type


In [12]:
#df_logs = df_logs.drop(columns=['_source.event_data.Company', '_source.event_data.CurrentDirectory'])

In [13]:
df_logs_dummies = pd.get_dummies(df_logs, dummy_na=True)

In [14]:
from sklearn.preprocessing import PowerTransformer
scaled_df_as_arrays = PowerTransformer().fit_transform(df_logs_dummies)
df_scaled = pd.DataFrame(scaled_df_as_arrays, index=df_logs_dummies.index, columns=df_logs_dummies.columns)

/home/cclousop-237/git_projects/virtual_envs/ds/lib/python3.6/site-packages/sklearn/preprocessing/data.py:2778: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [15]:
from sklearn.cluster import DBSCAN
import numpy as np

clustering = DBSCAN(min_samples=3).fit(df_logs_dummies)

In [16]:
np.max(clustering.labels_)

7

In [17]:
df_logs_dummies.shape

(266, 15)

In [18]:
from sklearn.cluster import AgglomerativeClustering
params_to_use = {'linkage': 'average', 'affinity': 'manhattan'}
clustering = AgglomerativeClustering(**params_to_use).fit(df_logs_dummies)
#print(params_to_use)

In [19]:
df_logs[clustering.labels_==1]

,_source.event_data.User,_source.event_data.Company
0,DEMO-OP1\support,Microsoft Corporation
1,DEMO-OP1\support,Microsoft Corporation
2,DEMO-OP1\support,Microsoft Corporation
8,DEMO-OP1\support,Microsoft Corporation
11,DEMO-OP1\support,Microsoft Corporation
12,DEMO-OP1\support,Microsoft Corporation
14,DEMO-OP1\support,Sysinternals - www.sysinternals.com
16,DEMO-OP1\support,Microsoft Corporation
25,DEMO-OP1\support,Microsoft Corporation
26,DEMO-OP1\support,Microsoft Corporation


## Better try

In [102]:
df_logs = df_logs_with_threat

In [95]:
pd.set_option('display.max_rows', None)

In [96]:
df_logs

,_id,_index,_score,_source.@timestamp,_source.beat.hostname,_source.beat.name,_source.beat.version,_source.computer_name,_source.event_data.CommandLine,_source.event_data.Company,_source.event_data.CurrentDirectory,_source.event_data.Description,_source.event_data.FileVersion,_source.event_data.Hashes,_source.event_data.Image,_source.event_data.IntegrityLevel,_source.event_data.LogonGuid,_source.event_data.LogonId,_source.event_data.ParentCommandLine,_source.event_data.ParentImage,_source.event_data.ParentProcessGuid,_source.event_data.ParentProcessId,_source.event_data.ProcessGuid,_source.event_data.ProcessId,_source.event_data.Product,_source.event_data.TerminalSessionId,_source.event_data.User,_source.event_data.UtcTime,_source.event_id,_source.host.name,_source.level,_source.log_name,_source.message,_source.opcode,_source.process_id,_source.provider_guid,_source.record_number,_source.source_name,_source.task,_source.thread_id,_source.type,_source.user.domain,_source.user.identifier,_source.user.name,_source.user.type,_source.version,_type
0,x-hoxWgBh5WDUxgRVGyf,winlogbeat-6.4.3-2019.02.07,2.486658,2019-02-07T00:42:53.617Z,demo-op1,demo-op1,6.4.3,demo-op1,c:\windows\system32\installagent.exe -embedding,Microsoft Corporation,c:\windows\system32\,InstallAgent,10.0.15063.0 (WinBuild.160101.0800),SHA256=1F340D262041D4D785964E4A74FB7C3BFBAEED18A3AD7E722284E8D9F68BBB92,C:\Windows\System32\InstallAgent.exe,Medium,{3CEC4195-5A8D-5C58-0000-0020B3860800},0x886b3,C:\Windows\system32\svchost.exe -k DcomLaunch,C:\Windows\System32\svchost.exe,{3CEC4195-59EB-5C58-0000-0010A4710000},788,{3CEC4195-7F0D-5C5B-0000-00102DEC1D01},6876,Microsoft® Windows® Operating System,1,DEMO-OP1\support,2019-02-07 00:42:53.613,1,demo-op1,Information,Microsoft-Windows-Sysmon/Operational,Process Create:\nRuleName: \nUtcTime: 2019-02-07 00:42:53.613\nProcessGuid: {3CEC4195-7F0D-5C5B-0000-00102DEC1D01}\nProcessId: 6876\nImage: C:\Windows\System32\InstallAgent.exe\nFileVersion: 10.0.15063.0 (WinBuild.160101.0800)\nDescription: InstallAgent\nProduct: Microsoft® Windows® Operating System\nCompany: Microsoft Corporation\nCommandLine: C:\Windows\System32\InstallAgent.exe -Embedding\nCurrentDirectory: C:\Windows\system32\\nUser: DEMO-OP1\support\nLogonGuid: {3CEC4195-5A8D-5C58-0000-0020B3860800}\nLogonId: 0x886B3\nTerminalSessionId: 1\nIntegrityLevel: Medium\nHashes: SHA256=1F340D262041D4D785964E4A74FB7C3BFBAEED18A3AD7E722284E8D9F68BBB92\nParentProcessGuid: {3CEC4195-59EB-5C58-0000-0010A4710000}\nParentProcessId: 788\nParentImage: C:\Windows\System32\svchost.exe\nParentCommandLine: C:\Windows\system32\svchost.exe -k DcomLaunch,Info,2996,{5770385F-C22A-43E0-BF4C-06F5698FFBD9},17935,Microsoft-Windows-Sysmon,Process Create (rule: ProcessCreate),4672,wineventlog,NT AUTHORITY,S-1-5-18,SYSTEM,User,5,doc
1,-uh0xWgBh5WDUxgRoXTT,winlogbeat-6.4.3-2019.02.07,2.486658,2019-02-07T00:56:20.119Z,demo-op1,demo-op1,6.4.3,demo-op1,c:\windows\system32\wuapihost.exe -embedding,Microsoft Corporation,c:\windows\system32\,wuapihost,10.0.15063.0 (WinBuild.160101.0800),SHA256=786958A30D21AFC93BD819075A04EB1E7893954776808790F1255FE7F3FBFB0A,C:\Windows\System32\wuapihost.exe,Medium,{3CEC4195-5A8D-5C58-0000-0020B3860800},0x886b3,C:\Windows\system32\svchost.exe -k DcomLaunch,C:\Windows\System32\svchost.exe,{3CEC4195-59EB-5C58-0000-0010A4710000},788,{3CEC4195-8234-5C5B-0000-001084461F01},5084,Microsoft® Windows® Operating System,1,DEMO-OP1\support,2019-02-07 00:56:20.114,1,demo-op1,Information,Microsoft-Windows-Sysmon/Operational,Process Create:\nRuleName: \nUtcTime: 2019-02-07 00:56:20.114\nProcessGuid: {3CEC4195-8234-5C5B-0000-001084461F01}\nProcessId: 5084\nImage: C:\Windows\System32\wuapihost.exe\nFileVersion: 10.0.15063.0 (WinBuild.160101.0800)\nDescription: wuapihost\nProduct: Microsoft® Windows® Operating System\nCompany: Microsoft Corporation\nCommandLine: C:\Windows\System32\wuapihost.exe -Embedding\nCurrentDirectory: C:\Windows\system32\\nUser: DEMO-OP1\support\nLogonGuid: {3CEC4195-5A8D

In [23]:
#df_logs.iloc[186]

In [24]:
df_logs = df_logs_without_threat

In [107]:
df_logs["_source.event_data.CommandLine"] = df_logs["_source.event_data.CommandLine"].str.lower()
df_logs["_source.event_data.CommandLine"] = df_logs["_source.event_data.CommandLine"].str.replace("\"","")
df_logs["_source.event_data.CurrentDirectory"] = df_logs["_source.event_data.CurrentDirectory"].str.lower()
df_logs = df_logs[["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"]]
df_logs_dummies = pd.get_dummies(df_logs, dummy_na=True)


In [26]:
df_logs_dummies.shape

(260, 110)

## DBSCAN

In [97]:
from sklearn.preprocessing import PowerTransformer
scaled_df_as_arrays = PowerTransformer().fit_transform(df_logs_dummies)
df_scaled = pd.DataFrame(scaled_df_as_arrays, index=df_logs_dummies.index, columns=df_logs_dummies.columns)

from sklearn.cluster import DBSCAN
import numpy as np

clustering = DBSCAN(eps=3).fit(df_scaled) # With eps=1.0 command lines are grouped given a directory. 
# With 1.5 grouped only based on directory

/home/cclousop-237/git_projects/virtual_envs/ds/lib/python3.6/site-packages/sklearn/preprocessing/data.py:2778: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [108]:
from sklearn.cluster import DBSCAN
import numpy as np

clustering = DBSCAN(eps=1.5).fit(df_logs_dummies) # With eps=1.0 command lines are grouped given a directory. 
# With 1.5 grouped only based on directory

In [109]:
np.max(clustering.labels_)

1

In [110]:
df_logs[clustering.labels_==-1]      

,_source.event_data.CommandLine,_source.event_data.CurrentDirectory
28,c:\windows\system32\notepad.exe c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\users\support\desktop\rinetd_exploit\
29,c:\windows\system32\notepad.exe c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\users\support\desktop\rinetd_exploit\
36,c:\windows\system32\notepad.exe s:\nabil_do_not_touch\launch_exploit.txt,s:\nabil_do_not_touch\
44,c:\windows\immersivecontrolpanel\systemsettings.exe -servername:microsoft.windows.immersivecontrolpanel,c:\windows\immersivecontrolpanel\
69,c:\windows\syswow64\runonce.exe /run6432,c:\windows\syswow64\
118,c:\windows\immersivecontrolpanel\systemsettings.exe -servername:microsoft.windows.immersivecontrolpanel,c:\windows\immersivecontrolpanel\
205,c:\windows\system32\notepad.exe s:\nabil_do_not_touch\launch_exploit.txt,s:\nabil_do_not_touch\
243,c:\program files (x86)\provconnect\provconnect device agent\device agent\bin\provconnect.notification.windows.exe,c:\windows\syswow64\


In [83]:
clustering.labels_[28]

-1

In [133]:
df_logs[clustering.labels_==2].drop(columns=["_source.message"])     

,_id,_index,_score,_source.@timestamp,_source.beat.hostname,_source.beat.name,_source.beat.version,_source.computer_name,_source.event_data.CommandLine,_source.event_data.Company,_source.event_data.CurrentDirectory,_source.event_data.Description,_source.event_data.FileVersion,_source.event_data.Hashes,_source.event_data.Image,_source.event_data.IntegrityLevel,_source.event_data.LogonGuid,_source.event_data.LogonId,_source.event_data.ParentCommandLine,_source.event_data.ParentImage,_source.event_data.ParentProcessGuid,_source.event_data.ParentProcessId,_source.event_data.ProcessGuid,_source.event_data.ProcessId,_source.event_data.Product,_source.event_data.TerminalSessionId,_source.event_data.User,_source.event_data.UtcTime,_source.event_id,_source.host.name,_source.level,_source.log_name,_source.opcode,_source.process_id,_source.provider_guid,_source.record_number,_source.source_name,_source.task,_source.thread_id,_source.type,_source.user.domain,_source.user.identifier,_source.user.name,_source.user.type,_source.version,_type


In [111]:
df_logs["_source.event_data.CommandLine"].str.split(".exe", n=1)

0                                                                                                                                                                                                                   [c:\windows\system32\installagent,  -embedding]
1                                                                                                                                                                                                                      [c:\windows\system32\wuapihost,  -embedding]
2                                                                                                                                                                   [c:\windows\updateassistant\updateassistant,  /clientid win10upgrade:vnl:nhv20:{} /calendarrun]
3                                                                                                                                                                                                                           

In [36]:
"aaa exe".split(".exe")

['aaa exe']

In [40]:
line = "aaa exe"
d = ".exe"
s =  [e+d for e in line.split(d) if e]
print(s)

['aaa exe.exe']


In [41]:
idx = 'i need toKeepTHISword'.index("THIS")
'i need toKeepTHISword'[idx:idx+len("THIS")]

'THIS'

In [42]:
idx

13

In [133]:
def splt_exe_and_parameter_involving_path(a):
    tokens = []
    fullstr = a
    substr_idx = -1

    while fullstr != "":
        try:
            substr_idx = fullstr.index(".exe")
        except:
            pass
        if substr_idx>0:
            #print(fullstr)
            substr_beg = fullstr[0:substr_idx+len(".exe")]
            substr_end = fullstr[substr_idx+len(".exe")+1:]
            tokens.append(substr_beg)
            #tokens.append(substr_end)
            fullstr = fullstr[substr_idx+len(".exe")+1:]
            #print("***"+fullstr)
            substr_idx = -1
        else:
            try:
                substr_idx = fullstr.index(":\\")
                parameter_path = fullstr[substr_idx-1:]
                #print("aaaaaaaaaaa", parameter_path)

                tokens.append(parameter_path)
                fullstr = ""
            except:
                fullstr = ""
    return tokens

In [134]:
df_temp = df_logs["_source.event_data.CommandLine"].apply(splt_exe_and_parameter_involving_path)

In [155]:
pd.set_option('display.max_colwidth',1000)
df_temp = pd.DataFrame(df_temp.apply(pd.Series))
df_temp = df_temp.rename(index=str, columns={0: "Command_EXE", 1: "Parameter_involving_folder"})
df_temp

,Command_EXE,Parameter_involving_folder
0,c:\windows\system32\installagent.exe,NaN
1,c:\windows\system32\wuapihost.exe,NaN
2,c:\windows\updateassistant\updateassistant.exe,NaN
3,c:\windows\system32\compattelrunner.exe,NaN
4,c:\windows\system32\compattelrunner.exe,NaN
5,c:\windows\system32\installagentuserbroker.exe,NaN
6,c:\windows\system32\wuapihost.exe,NaN
7,c:\windows\system32\compattelrunner.exe,NaN
8,c:\windows\system32\devicecensus.exe,NaN
9,c:\windows\system32\compattelrunner.exe,NaN


In [153]:
def get_path_from_exe(full_path):
    path_elements = str(full_path).split("\\")
    return "\\".join(path_elements[:-1])

In [228]:
def get_exe_from_full_path(full_path):
    path_elements = str(full_path).split("\\")
    return path_elements[-1:][0]

In [229]:
df_temp["Path_to_EXE"] = df_temp["Command_EXE"].apply(get_path_from_exe)

In [230]:
df_temp["EXE"] = df_temp["Command_EXE"].apply(get_exe_from_full_path)

In [231]:
df_temp

,Command_EXE,Parameter_involving_folder,Path_to_EXE,EXE
0,c:\windows\system32\installagent.exe,NaN,c:\windows\system32,installagent.exe
1,c:\windows\system32\wuapihost.exe,NaN,c:\windows\system32,wuapihost.exe
2,c:\windows\updateassistant\updateassistant.exe,NaN,c:\windows\updateassistant,updateassistant.exe
3,c:\windows\system32\compattelrunner.exe,NaN,c:\windows\system32,compattelrunner.exe
4,c:\windows\system32\compattelrunner.exe,NaN,c:\windows\system32,compattelrunner.exe
5,c:\windows\system32\installagentuserbroker.exe,NaN,c:\windows\system32,installagentuserbroker.exe
6,c:\windows\system32\wuapihost.exe,NaN,c:\windows\system32,wuapihost.exe
7,c:\windows\system32\compattelrunner.exe,NaN,c:\windows\system32,compattelrunner.exe
8,c:\windows\system32\devicecensus.exe,NaN,c:\windows\system32,devicecensus.exe
9,c:\windows\system32\compattelrunner.exe,NaN,c:\windows\system32,compattelrunner.exe


In [158]:
df_dummies = df_temp[["Path_to_EXE", "Parameter_involving_folder", "EXE"]]
df_dummies_advanced = pd.get_dummies(df_dummies, dummy_na=True)

In [233]:
from sklearn.cluster import DBSCAN
import numpy as np

clustering = DBSCAN(eps=1.2).fit(df_dummies_advanced) # With eps=1.0 command lines are grouped given a directory. 
# With 1.5 grouped only based on directory

In [234]:
np.max(clustering.labels_)

5

In [236]:
df_temp[clustering.labels_==-1]

,Command_EXE,Parameter_involving_folder,Path_to_EXE,EXE
13,c:\program files\rempl\sedlauncher.exe,NaN,c:\program files\rempl,sedlauncher.exe
22,c:\users\support\appdata\local\temp\b764172d-95d3-49e3-b6e2-babae644e3c9\dismhost.exe,NaN,c:\users\support\appdata\local\temp\b764172d-95d3-49e3-b6e2-babae644e3c9,dismhost.exe
25,c:\users\support\appdata\local\temp\f679bb89-2e54-4843-8ae7-1aba9ebbba6e\dismhost.exe,NaN,c:\users\support\appdata\local\temp\f679bb89-2e54-4843-8ae7-1aba9ebbba6e,dismhost.exe
27,c:\windows\winsxs\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.15063.1111_none_b9580e706f668ab0\tiworker.exe,NaN,c:\windows\winsxs\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.15063.1111_none_b9580e706f668ab0,tiworker.exe
28,c:\windows\system32\notepad.exe,c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\windows\system32,notepad.exe
29,c:\windows\system32\notepad.exe,c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\windows\system32,notepad.exe
30,c:\users\support\desktop\rinetd_exploit\rinetd.exe,c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\users\support\desktop\rinetd_exploit,rinetd.exe
31,c:\windows\system32\rundll32.exe,"c:\windows\system32\firewallcontrolpanel.dll,shownotificationdialog /configure /etonly 0 /onprofiles 6 /otherallowed 0 /otherblocked 0 /otheredgeallowed 0 /newblocked 2 c:\users\support\desktop\rinetd_exploit\rinetd.exe",c:\windows\system32,rundll32.exe
32,c:\users\support\desktop\rinetd_exploit\rinetd.exe,c:\users\support\desktop\rinetd\rinetd.conf,c:\users\support\desktop\rinetd_exploit,rinetd.exe
36,c:\windows\system32\notepad.exe,s:\nabil_do_not_touch\launch_exploit.txt,c:\windows\system32,notepad.exe


## Agglomerative

In [41]:
from sklearn.cluster import AgglomerativeClustering
params_to_use = {'linkage': 'average', 'affinity': 'manhattan'}
clustering = AgglomerativeClustering(**params_to_use).fit(df_logs_dummies)
#print(params_to_use)

In [42]:
np.max(clustering.labels_)

1

In [44]:
df_logs[clustering.labels_==0]      

,_source.event_data.CommandLine,_source.event_data.CurrentDirectory
28,c:\windows\system32\notepad.exe c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\users\support\desktop\rinetd_exploit\
29,c:\windows\system32\notepad.exe c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\users\support\desktop\rinetd_exploit\
36,c:\windows\system32\notepad.exe s:\nabil_do_not_touch\launch_exploit.txt,s:\nabil_do_not_touch\
39,\systemroot\system32\smss.exe 000000e4 00000080,c:\windows\
44,c:\windows\immersivecontrolpanel\systemsettings.exe -servername:microsoft.windows.immersivecontrolpanel,c:\windows\immersivecontrolpanel\
45,\systemroot\system32\smss.exe 000000f8 00000080,c:\windows\
55,\systemroot\system32\smss.exe 000000f8 000000bc,c:\windows\
59,\systemroot\system32\smss.exe 000000d0 000000bc,c:\windows\
69,c:\windows\syswow64\runonce.exe /run6432,c:\windows\syswow64\
118,c:\windows\immersivecontrolpanel\systemsettings.exe -servername:microsoft.windows.immersivecontrolpanel,c:\windows\immersivecontrolpanel\


## Think different ≠

### <font color='blue'> iso datasets Loic (with all 2 logs: fit without threat, predict on dataset including the threat) </font>

In [15]:
df1 = df_logs_without_threat[["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"]]
df2 = df_logs_with_threat[["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"]]
pd.set_option('display.max_colwidth',1000)
pd.concat([df1, df2]).drop_duplicates(keep=False)

,_source.event_data.CommandLine,_source.event_data.CurrentDirectory
30,c:\users\support\desktop\rinetd_exploit\rinetd.exe -c c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\windows\system32\
31,"c:\windows\system32\rundll32.exe c:\windows\system32\firewallcontrolpanel.dll,shownotificationdialog /configure /etonly 0 /onprofiles 6 /otherallowed 0 /otherblocked 0 /otheredgeallowed 0 /newblocked 2 c:\users\support\desktop\rinetd_exploit\rinetd.exe",c:\windows\system32\


In [16]:
diff = set(zip(df2["_source.event_data.CommandLine"], df2["_source.event_data.CurrentDirectory"])) - set(zip(df1["_source.event_data.CommandLine"], df1["_source.event_data.CurrentDirectory"]))


In [17]:
pd.DataFrame(list(diff), columns=["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"])

,_source.event_data.CommandLine,_source.event_data.CurrentDirectory
0,c:\users\support\desktop\rinetd_exploit\rinetd.exe -c c:\users\support\desktop\rinetd\rinetd.conf,c:\windows\system32\
1,c:\windows\system32\notepad.exe c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\users\support\desktop\rinetd_exploit\
2,"c:\windows\system32\rundll32.exe c:\windows\system32\firewallcontrolpanel.dll,shownotificationdialog /configure /etonly 0 /onprofiles 6 /otherallowed 0 /otherblocked 0 /otheredgeallowed 0 /newblocked 2 c:\users\support\desktop\rinetd_exploit\rinetd.exe",c:\windows\system32\
3,c:\users\support\desktop\rinetd_exploit\rinetd.exe -c c:\users\support\desktop\rinetd_exploit\rinetd.conf,c:\windows\system32\


In [68]:
df_logs_other_day["_source.event_data.CommandLine"] = df_logs_other_day["_source.event_data.CommandLine"].str.lower()
df_logs_other_day["_source.event_data.CommandLine"] = df_logs_other_day["_source.event_data.CommandLine"].str.replace("\"","")
df_logs_other_day["_source.event_data.CurrentDirectory"] = df_logs_other_day["_source.event_data.CurrentDirectory"].str.lower()

### <font color='blue'>  with the 2 separate logs

In [67]:
#res = es.get(index="gebd-tracks-2018.12.07", doc_type='doc')
#res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={'query':{'match_all':{}}, "size": 10000})
res = es.search(index='winlogbeat-6.4.3-2019.02.13',body={ \
            "query": {
                "bool": {
                    "must": [
                      {
                        "match_phrase": {
                          "log_name": {
                            "query": "Microsoft-Windows-Sysmon/Operational"
                          }
                        }
                      },
                      {
                        "match_phrase": {
                          "task": {
                            "query": "Process Create (rule: ProcessCreate)"
                          }
                        }
                      }
                    ]
                }
            }, "size": 10000})

#print(res['hits'])

#df = pd.DataFrame.from_dict(res['hits']['hits'])
from pandas.io.json import json_normalize
df_logs_other_day = json_normalize(res['hits']['hits'])
pd.set_option('display.max_columns', None)

print("--- Column names and shape LOGS", df_logs_other_day.columns, df_logs_other_day.shape)
print(df_logs_other_day.head())
print("SHAPE:", df_logs_other_day.shape)



--- Column names and shape LOGS Index(['_id', '_index', '_score', '_source.@timestamp',
       '_source.beat.hostname', '_source.beat.name', '_source.beat.version',
       '_source.computer_name', '_source.event_data.CommandLine',
       '_source.event_data.Company', '_source.event_data.CurrentDirectory',
       '_source.event_data.Description', '_source.event_data.FileVersion',
       '_source.event_data.Hashes', '_source.event_data.Image',
       '_source.event_data.IntegrityLevel', '_source.event_data.LogonGuid',
       '_source.event_data.LogonId', '_source.event_data.ParentCommandLine',
       '_source.event_data.ParentImage',
       '_source.event_data.ParentProcessGuid',
       '_source.event_data.ParentProcessId', '_source.event_data.ProcessGuid',
       '_source.event_data.ProcessId', '_source.event_data.Product',
       '_source.event_data.TerminalSessionId', '_source.event_data.User',
       '_source.event_data.UtcTime', '_source.event_id', '_source.host.name',
       '_sour

In [69]:
df3 = df_logs_other_day[["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"]]

In [70]:
diff = set(zip(df2["_source.event_data.CommandLine"], df2["_source.event_data.CurrentDirectory"])) - set(zip(df3["_source.event_data.CommandLine"], df3["_source.event_data.CurrentDirectory"]))


In [71]:
pd.DataFrame(list(diff), columns=["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"])

,_source.event_data.CommandLine,_source.event_data.CurrentDirectory
0,\systemroot\system32\smss.exe 00000080 000000bc,c:\windows\
1,c:\windows\system32\ctfmon.exe /n,c:\windows\system32\
2,c:\windows\system32\compattelrunner.exe -m:generaltel.dll -f:rungeneraltelemetry -cv hoou/aa0v0cf7hdi.1.2 -sendfulltelemetry -throttleutc,c:\windows\system32\
3,logonui.exe /flags:0x0 /state0:0xa3d7e855 /state1:0x41c64e6d,c:\windows\system32\
4,c:\program files\rempl\sedlauncher.exe,c:\windows\system32\
5,c:\windows\system32\compattelrunner.exe -m:appraiser.dll -f:updateavstatus -cv:2aznly/on0smy5lt.3,c:\windows\system32\
6,c:\programdata\microsoft\windows defender\platform\4.18.1812.3-0\mpcmdrun.exe getdeviceticket -accesskey c8539b5c-3ef8-8b98-aa11-56a32409ea04,c:\windows\system32\
7,c:\windows\system32\werfault.exe -u -p 3396 -s 1296,c:\windows\system32\
8,c:\windows\system32\defrag.exe -p ba4 -s 00000000000003a0 -b c:,c:\windows\system32\
9,c:\windows\system32\defrag.exe -c -h -o -$,c:\windows\system32\


In [186]:
res = pd.concat([df3, df2]).drop_duplicates(keep=False)
res = pd.DataFrame(res, columns=["_source.event_data.CommandLine", "_source.event_data.CurrentDirectory"])
res[res["_source.event_data.CommandLine"].str.contains("logonui")]
res.drop(res[res["_source.event_data.CommandLine"].str.contains("logonui")].index)
#res
#res["_source.event_data.CommandLine"]

,_source.event_data.CommandLine,_source.event_data.CurrentDirectory
7,wmiadap.exe /f /t /r,c:\windows\system32\
8,c:\windows\system32\defrag.exe -p b30 -s 0000000000000204 -b c:,c:\windows\system32\
12,c:\windows\system32\srtasks.exe executescheduledsppcreation,c:\windows\system32\
23,c:\windows\system32\compattelrunner.exe -m:invagent.dll -f:runupdate -cv:qtpv0is1dui2qv2v.5,c:\windows\system32\
38,\systemroot\system32\smss.exe 000000d8 00000080,c:\windows\
64,\systemroot\system32\smss.exe 000000b4 00000080,c:\windows\
123,c:\windows\system32\notepad.exe c:\users\support\desktop\test.punch.txt,c:\users\support\desktop\
163,c:\windows\system32\mmc.exe c:\windows\system32\services.msc,c:\windows\system32\
254,\systemroot\system32\smss.exe 000000e0 00000080,c:\windows\
260,\systemroot\system32\smss.exe 000000b0 00000080,c:\windows\


## JUNK BELOW

In [ ]:
#res = es.get(index="gebd-tracks-2018.12.07", doc_type='doc')
#res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={'query':{'match_all':{}}, "size": 10000})
res = es.search(index='winlogbeat-6.4.3-2019.02.07',body={ \
            "query": { \
                "bool": { \
                    "must": [ \
                      { \
                        "match_phrase": { \
                          "log_name": { \
                            "query": "Microsoft-Windows-Sysmon/Operational" \
                          } \
                        } \
                      }, \
                      { \
                        "match_phrase": { \
                          "task": { \
                            "query": "Process Create (rule: ProcessCreate)" \
                          } \
                        } \
                      } \
                    ], \
                    "filter": [ \
                      { \
                        "bool": { \
                          "must_not": { \
                            "multi_match": { \
                              "type": "best_fields", \
                              "query": "rinetd_exploit", \
                              "lenient": True \
                            } \
                          } \
                        } \
                      }, \
                      { \
                        "match_all": {} \
                      } \
                    ] \
                } \
            }, "size": 10000})

#print(res['hits'])

#df = pd.DataFrame.from_dict(res['hits']['hits'])
from pandas.io.json import json_normalize
df_logs_without_threat = json_normalize(res['hits']['hits'])
pd.set_option('display.max_columns', None)

print("--- Column names and shape LOGS", df_logs.columns, df_logs.shape)
print(df_logs.head())
print("SHAPE:", df_logs.shape)

A voir dans les données :
Avec Kibana, les occurences de rinetd (création fichiers, scénario attaque)

In [87]:
np.unique(df_logs_other_day["_source.event_data.CurrentDirectory"])

array(['c:\\users\\support\\', 'c:\\users\\support\\desktop\\',
       'c:\\windows\\', 'c:\\windows\\immersivecontrolpanel\\',
       'c:\\windows\\system32\\', 'c:\\windows\\syswow64\\'], dtype=object)

In [88]:
np.unique(df_logs_without_threat["_source.event_data.CurrentDirectory"])

array(['c:\\windows\\', 'c:\\windows\\immersivecontrolpanel\\',
       'c:\\windows\\system32\\', 'c:\\windows\\syswow64\\',
       's:\\nabil_do_not_touch\\'], dtype=object)

In [89]:
np.unique(df_logs_with_threat["_source.event_data.CurrentDirectory"])

array(['c:\\users\\support\\desktop\\rinetd_exploit\\', 'c:\\windows\\',
       'c:\\windows\\immersivecontrolpanel\\', 'c:\\windows\\system32\\',
       'c:\\windows\\syswow64\\', 's:\\nabil_do_not_touch\\'],
      dtype=object)

Solution diff with index for stop words, allowing logonui* for instance